In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import torch

CUDA_VISIBLE_DEVICES = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = CUDA_VISIBLE_DEVICES

gender_info = {
    "P1": 0, "P3": 0, "P4": 0, "P7": 0, "P8": 0, "P10": 0, "P13": 0, "P15": 0,
    "P2": 1, "P5": 1, "P6": 1, "P9": 1, "P11": 1, "P12": 1, "P14": 1, "P16": 1,
    "P17": 1, "P18": 1, "P19": 1, "P20": 1
}


# Loading A small Demo dataset with RGB, Depth, RPC, RT and GT Mesh modalities...

In [2]:
from dataset.dataset_mmMesh2_vis import RF3DPoseDataset, ToTensor
import torchvision.transforms as transforms

dataset_vis = RF3DPoseDataset([], transform=ToTensor(), load_save=True, use_image=True, is_demo=True, cache_dir="../cached_data_test_vis/")

Dataset loaded Successfully from ../cached_data_test_vis/rf3dpose_all ...
Kept 5000 / 5000 samples after filtering.


In [4]:
from dataset.data_loader_Plotting_projection import plot_frames_for_gif_depth
for i in range(10):#50
    plot_frames_for_gif_depth(dataset_vis, combined_gif_path=f'vis_depth/combined_output_depth_{i+1}.gif', act_id=i+1, is_plot="save")

Generating 5000 frames starting from index 0 for combined GIF with 4 columns...
tensor([  2,   1, 100]) image: torch.Size([3, 480, 640]), depth: torch.Size([1, 480, 640]), radar_tensor: torch.Size([121, 111, 31]), radar_points: torch.Size([1000, 4])
tensor([  2,   1, 101]) image: torch.Size([3, 480, 640]), depth: torch.Size([1, 480, 640]), radar_tensor: torch.Size([121, 111, 31]), radar_points: torch.Size([1000, 4])
tensor([  2,   1, 102]) image: torch.Size([3, 480, 640]), depth: torch.Size([1, 480, 640]), radar_tensor: torch.Size([121, 111, 31]), radar_points: torch.Size([1000, 4])
tensor([  2,   1, 103]) image: torch.Size([3, 480, 640]), depth: torch.Size([1, 480, 640]), radar_tensor: torch.Size([121, 111, 31]), radar_points: torch.Size([1000, 4])
tensor([  2,   1, 104]) image: torch.Size([3, 480, 640]), depth: torch.Size([1, 480, 640]), radar_tensor: torch.Size([121, 111, 31]), radar_points: torch.Size([1000, 4])
tensor([  2,   1, 105]) image: torch.Size([3, 480, 640]), depth: torch

KeyboardInterrupt: 

# Loading Full dataset with only radar-based RPC, RT and GT Mesh modalities...

### dataset param:

1. Split: train, test, val

2. main_modality: selected modality: default rt (No need to change, both provided)

3. protocol_id: (controlling the size of dataset)

Here is different from protocol in paper [IP, SIP, NIP], these protocol performance will be printed during main training. Can adjust to accomodate your expected training time!!

p1: 100% dataset, p2: 50% dataset, p3: 25% dataset

4. split_id: (controlling split)

Same as paper. s1: random split; s2: cross-subject split; s3: cross-action split

5. Temporal_window: sliding window size for concat adjacent radar frame.

Default: 4

6. Load_save: loading flag for preprocessed dataset. Better set to True, haven't test False can process from scratch or not.

7. Cache_dir: saved preprocessed dataset path. [***!!! input your own dir !!!***]






In [19]:
from dataset.dataset_mmMesh2 import RF3DPoseDataset, ToTensor
import torchvision.transforms as transforms

dataset_train = RF3DPoseDataset([], 
                                # Train-Test Specifics
                                split = "train",  
                                main_modality="rt", 
                                protocol_id="p1", 
                                split_id="s1", 
                                temporal_window=6,
                                # meta info
                                load_save=True, 
                                cache_dir="../mmDataset/MR-Mesh/",  
                                transform=transforms.Compose([ToTensor()]), 
                                )
dataset_test = RF3DPoseDataset([], 
                               # Train-Test Specifics
                               split = "test",  
                               main_modality="rt", 
                               protocol_id="p1", 
                               split_id="s1", 
                               temporal_window=6,
                               # meta info
                               load_save=True, 
                               cache_dir="../mmDataset/MR-Mesh/",  
                               transform=transforms.Compose([ToTensor()]), 
                               )

Dataset loaded Successfully from ../mmDataset/MR-Mesh/rf3dpose_all ...
Load indices from pre-saved ../mmDataset/MR-Mesh/rf3dpose_all/indeces.pkl.gz indices splits.
Loaded train Dataset with length 495512.
Unique sub in train: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Unique act in train: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
Kept 495506 / 495512 samples after filtering.
Dataset loaded Successfully from ../mmDataset/MR-Mesh/rf3dpose_all ...
Load indices from pre-saved ../mmDataset/MR-Mesh/rf3dpose_all/indeces.pkl.gz indices splits.
Loaded test Dataset with length 131809.
Unique sub in test: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Unique act in test: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,

### Dataloader getitem: 
Dict of components consist of Radar Modality[RT / RPC], Sample ID, GT information [Param, Calibration, bounding box, 2D projected mesh in RGB frame], RGBD modality (TODO)

### Details:
Sample ID: [indicator] [Sub_id, Act_id, Frame_id];

Radar RT: [rawImage_XYZ], (T_window, Azim, Depth, Elev), T_window: default 4;

Radar RPC: [radar_points], (T_window, N, 4), N = 1000 with zero-padding;

GT mesh: [vertices]; GT bbbox: [bbbox]; GT param: [parameter]; 

GT calib: [calibration]; GT tracking loc: [joint_root]: coord of tracking joint 0. 

GT 2D mesh proj: [projected_vertices] 








In [18]:
sample = dataset_train[0]
print("Sample ID:", sample.get('indicator', 'N/A'))
print("Sample keys:", sample.keys())
print("\n== Sample Content ==\n")

for i,key in zip(range(len(sample)), sample):
    print(f"--- Key {i+1}/{len(sample)} ---")
    if isinstance(sample[key], torch.Tensor):
        print(f"{key}: shape {sample[key].shape}, dtype {sample[key].dtype}")
    elif isinstance(sample[key], dict):
        for subkey in sample[key]:
            if isinstance(sample[key][subkey], torch.Tensor):
                print(f"{key}->{subkey}: shape {sample[key][subkey].shape}, dtype {sample[key][subkey].dtype}")
            else:
                print(f"{key}->{subkey}: type {type(sample[key][subkey])}")
    else:
        print(f"{key}: type {type(sample[key])}")
    

Sample ID: tensor([ 1, 10,  1])
Sample keys: dict_keys(['rawImage_XYZ', 'vertices', 'bbbox', 'projected_vertices', 'parameter', 'calibration', 'indicator', 'radar_points', 'joints_root'])

== Sample Content ==

--- Key 1/9 ---
rawImage_XYZ: shape torch.Size([6, 121, 111, 31]), dtype torch.float32
--- Key 2/9 ---
vertices: shape torch.Size([10475, 3]), dtype torch.float32
--- Key 3/9 ---
bbbox->pc: type <class 'dict'>
bbbox->tensor: type <class 'dict'>
--- Key 4/9 ---
projected_vertices: shape torch.Size([10475, 2]), dtype torch.float32
--- Key 5/9 ---
parameter->joints: shape torch.Size([55, 3]), dtype torch.float32
parameter->betas: shape torch.Size([10]), dtype torch.float32
parameter->pose_body: shape torch.Size([63]), dtype torch.float32
parameter->trans: shape torch.Size([3]), dtype torch.float32
parameter->root_orient: shape torch.Size([3]), dtype torch.float32
parameter->gender: shape torch.Size([1]), dtype torch.float32
--- Key 6/9 ---
calibration->cam_intrinsic: shape torch.Si